# RFM-анализ клиентской базы аптечной сети

Короткое саммари

Здесь должно быть концептуальное описание ваших действий:

- как выбирали метрики для анализа
- почему именно такие метрики
- какие ваши основные гипотезы
- ваш ход рассуждений
- Содержательная часть

Здесь будет непосредственно ваша аналитика:

- словесное описание запроса и цель, которую хотите достичь
- код запроса
- описание полученных результатов и промежуточные выводы
- любая другая важная на ваш взгляд информация
- И так для каждого запроса.

Выводы

Здесь должны быть финальные выводы:


- что у вас получилось
- подтвердились ли ваши гипотезы
- какие бизнес-решения вы предлагаете принять и почему

Важно: Из вашего отчета все члены команды 
(руководство, другие аналитики, менеджеры) должны понять - как им развивать компанию дальше.

Основной мотив исследования такой: скоро сезон спадет и выручки в аптеке начнут падать. У ребят есть механизм СМС-рассылок, но они не хотят тратить бюджеты впустую - они хотят провести персонализированную рассылку.

Идея такая: нет смысла предлагать персональную скидку для повышения среднего чека тем, кто вообще покупал один раз и то аскорбинку.

Соответственно, цели исследования:


- Провести классификацию пользователей

- Сформировать рекомендации - какой группе пользователей что предлагать

- Для решения этой задачи вам предлагается базово сосредоточится на RFM-анализе.

Основная проблема при проведении такого исследования - подобрать оптимальные пороги. Например, каких клиентов относить в группу 1 по признаку Recency, а кого в группу 2. И здесь мы оставляем вам пространство для  ̶к̶р̶е̶а̶т̶и̶в̶а̶ исследования. Вы сами будете выбирать методы, с помощью которых определите этот порог. Мы можем лишь посоветовать:

- Не забывайте про ABC
- Средние, медианы и перцентили хорошо показывают распределение той или иной величины
- С помощью графика boxplot можно также оценить распределение величин
- Самое главное - принимайте решение обоснованно, опираясь на данные, а не потому что «так захотелось».



* Recency — давность
* Frequency — частота
* Monetary - деньги

Для начала перед проведением самого RFM анализа стоит выбрать метрики:
для отбора товаров значительно важных по прибыли и количеству мы возьмем ABC анализ  
для разбиения под категорий в RFM анализе, возьмем медиану

1. F=3 приносит больше выручки, чем F=1 → (высокая частота = выручка).
2. M=3 покупают больше A-товаров.
3. M=1 — C-товары.
4. F=3 → больше позиций в чеке.
5. Сегмент 3-3-3 генерирует непропорционально высокую выручку.
6. R=3 чаще покупать B-товары (дополняющие покупки).
7. Клиенты с высоким Monetary получают больше скидок.
8. Клиенты из разных аптек отличаются по F и M.
9. A-товары дают основную выручку → подтверждение ABC.
10. C-товары часто покупаются разовыми клиентами → подтверждение поведения.

поля для работы 

select
--	*
	b.card 
	, s.dr_dat 
	, s.dr_tim 
	, s.dr_nchk 
	, s.dr_ndoc
	, null
	, s.dr_ndrugs 
	, s.dr_kol 
	, s.dr_czak 
	, s.dr_croz 
	, s.dr_sdisc 
from
	sales s 
join bonuscheques b on s.dr_ndoc = split_part(b.doc_id, '#', 2)::int 

-- revenue_abc 

with agg_abc as (
select
	dr_ndrugs
	, sum(dr_kol) as amount
	, sum(dr_kol * (dr_croz - dr_czak) - dr_sdisc) as profit
	, sum(dr_kol * dr_croz - dr_sdisc) as revenue
from
	sales
group by 
		dr_ndrugs
)
select
	dr_ndrugs as product
	, case
		when sum(revenue) over(order by revenue desc) / sum(revenue) over() * 100.0 <= 80 then 'A' 
		when sum(revenue) over(order by revenue desc) / sum(revenue) over() * 100.0 <= 95 then 'B'
		else 'C'
	end as revenue_abc 
from
	agg_abc
order by
	revenue_abc ;

select
--	*
	b.card 
	, now() - max(s.dr_dat) as R
	, count(distinct s.dr_ndoc) as F
	, sum(dr_kol * dr_croz - dr_sdisc) as M
from
	sales s 
join bonuscheques b on s.dr_ndoc = split_part(b.doc_id, '#', 2)::int 
group by 
	b.card
order by
	R DESC

есть аптечная сеть

После выбора задачи, вам нужно будет провести масштабное исследование по выбранной теме и подготовить бизнес-отчет с результатами. Отчет должен в себя включать:

Короткое саммари

Здесь должно быть концептуальное описание ваших действий:

как выбирали метрики для анализа
почему именно такие метрики
какие ваши основные гипотезы
ваш ход рассуждений
Содержательная часть

Здесь будет непосредственно ваша аналитика:

словесное описание запроса и цель, которую хотите достичь
код запроса
описание полученных результатов и промежуточные выводы
любая другая важная на ваш взгляд информация
И так для каждого запроса.

вот мы с тобой как то составляли гипотезы 
5. Сегмент 3-3-3 генерирует непропорционально высокую выручку.
6. R=3 чаще покупать B-товары (дополняющие покупки).
7. Клиенты с высоким Monetary получают больше скидок.
8. Клиенты из разных аптек отличаются по F и M.
9. A-товары дают основную выручку → подтверждение ABC.
10. C-товары часто покупаются разовыми клиентами → подтверждение поведения.

 есть задание Основной мотив исследования такой: скоро сезон спадет и выручки в аптеке начнут падать. У ребят есть механизм СМС-рассылок, но они не хотят тратить бюджеты впустую - они хотят провести персонализированную рассылку. Идея такая: нет смысла предлагать персональную скидку для повышения среднего чека тем, кто вообще покупал один раз и то аскорбинку. Соответственно, цели исследования: Провести классификацию пользователей Сформировать рекомендации - какой группе пользователей что предлагать Для решения этой задачи вам предлагается базово сосредоточится на RFM-анализе. Основная проблема при проведении такого исследования - подобрать оптимальные пороги. Например, каких клиентов относить в группу 1 по признаку Recency, а кого в группу 2. И здесь мы оставляем вам пространство для ̶к̶р̶е̶а̶т̶и̶в̶а̶ исследования. Вы сами будете выбирать методы, с помощью которых определите этот порог. Мы можем лишь посоветовать: Не забывайте про ABC Средние, медианы и перцентили хорошо показывают распределение той или иной величины С помощью графика boxplot можно также оценить распределение величин Самое главное - принимайте решение обоснованно, опираясь на данные, а не потому что «так захотелось». 

есть данные за период 
2022-05-01	2022-06-09

есть данные таблица sales название столбцов dr_dat dr_tim dr_nchk dr_ndoc dr_apt dr_kkm dr_tdoc dr_tpay dr_cdrugs dr_ndrugs dr_suppl dr_prod dr_kol dr_czak dr_croz dr_sdisc dr_cdisc dr_bcdisc dr_tabempl dr_pos dr_vzak пример данных 2022-05-01 08:33:09 6251 7001554 7 22553 Розничная реализация 18 20398 ОТРИВИН 0,1% 10МЛ. №1 НАЗАЛ.СПРЕЙ ФЛ. /ГЛАКСО/НОВАРТИС/ Фармкомплект ООО Новартис Консьюмер Хелс С.А. 1.0 146.0 175.0 8.0 9 200010024843 37 1 1 2022-05-01 09:03:32 4659 6002284 6 22568 Розничная реализация 18 541023 РАСТОРОПШИ ШРОТ 100Г. ПОР. /ИВАН ДА МАРЬЯ/ Катрен г.Химки ИВАН ДА МАРЬЯ ООО 2.0 38.62 67.0 16.0 35 200000000022 27 1 1 2022-05-01 09:07:49 6077 2002811 2 22569 Розничная реализация 18 25634 АККУ-ЧЕК ТЕСТ-ПОЛОСКИ АКТИВ №50 [ACCU-CHEK] Катрен г.Химки РОШ ДИАБЕТС КЕА ГМБХ 1.0 1107.57 1363.0 0.0 NULL NULL 25 1 1 2022-05-01 09:07:49 6077 2002811 2 22569 Розничная реализация 18 1504331 Карта LOYALITY 0,01Р ООО "Континент" РОШ ДИАБЕТС КЕА ГМБХ 1.0 0.01 0.01 0.0 NULL NULL 25 2 1 2022-05-01 09:08:38 6253 7001554 7 22553 Розничная реализация 18 25975 ЛИНКАС ЛОР ЛИМОН+МЕД №16 ПАСТИЛКИ Авеста HERBION ПАКИСТАН 1.0 122.42 173.0 12.0 9 200010023454 37 1 1 таблица bonuscheques название столбцов datetime shop card bonus_earned bonus_spent summ summ_with_disc doc_id пример данных 2021-07-13 12:56:09.000 Аптека 2 2000200195023 51 0 3400 3400 15#2002741#65938#2_29 2021-07-30 10:42:00.000 Аптека 2 2000200193494 57 0 747 747 15#2002972#65955#2_5 2021-10-11 12:55:23.000 Аптека 2 2000200199106 92 253 3077 3077 15#2004060#66028#2_29 2021-10-14 14:48:56.000 Аптека 2 2000200168768 1 0 54 54 15#2004107#66031#2_57 уже сделанный abc -- revenue_abc with agg_abc as ( select dr_ndrugs , sum(dr_kol) as amount , sum(dr_kol * (dr_croz - dr_czak) - dr_sdisc) as profit , sum(dr_kol * dr_croz - dr_sdisc) as revenue from sales group by dr_ndrugs ) select dr_ndrugs as product , case when sum(revenue) over(order by revenue desc) / sum(revenue) over() * 100.0 <= 80 then 'A' when sum(revenue) over(order by revenue desc) / sum(revenue) over() * 100.0 <= 95 then 'B' else 'C' end as revenue_abc from agg_abc order by revenue_abc ; предварительно сделанный RFM select -- * b.card , now() - max(s.dr_dat) as R , count(distinct s.dr_ndoc) as F , sum(dr_kol * dr_croz - dr_sdisc) as M from sales s join bonuscheques b on s.dr_ndoc = split_part(b.doc_id, '#', 2)::int group by b.card order by R DESC

поэтапно., у меня мало времени все надо очень продумать и сильно не усложнять, в итоге я эту работу потом оформлю сам в docs